## Import

In [383]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import train_test_split

sns.set_style("darkgrid")

### Importare dataset

In [467]:
df1 = pd.read_csv('./combined_data_1.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])

df1['Rating'] = df1['Rating'].astype(float)

print('Dataset 1 shape:',df1.shape)

# Mi genera un nuovo indice da 1
df1 = df1.reset_index()
print('Dataset 1 shape without rating null:',df1.shape)

print('-Dataset examples-')
print(df1[:5])


Dataset 1 shape: (24058263, 2)
Dataset 1 shape without rating null: (24058263, 3)
-Dataset examples-
   index  Cust_Id  Rating
0      0       1:     NaN
1      1  1488844     3.0
2      2   822109     5.0
3      3   885013     4.0
4      4    30878     4.0


### Prendo i primi x

In [468]:
df1 = df1.head(1000)

### df_nan è un DataFrame che indica True per le righe in cui la colonna 'Rating' è nulla e False altrimenti. Quindi, ogni volta che c'è una transizione da False a True nella colonna 'Rating', rappresenta l'inizio di un nuovo "film"

In [469]:
df_nan = pd.DataFrame(pd.isnull(df1.Rating))
df_nan = df_nan[df_nan['Rating'] == True]
df_nan = df_nan.reset_index()

movie_np = []
movie_id = 1

for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
    temp = np.full((1,i-j-1), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1

# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(df1) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)

#print('Movie numpy:',movie_np)
#print('Length:',len(movie_np))

df = df1[pd.notnull(df1['Rating'])]

df['Movie_Id'] = movie_np.astype(int)
df['Cust_Id'] = df['Cust_Id'].astype(int)
print('-Dataset examples-')
print(df)


-Dataset examples-
     index  Cust_Id  Rating  Movie_Id
1        1  1488844     3.0         1
2        2   822109     5.0         1
3        3   885013     4.0         1
4        4    30878     4.0         1
5        5   823519     3.0         1
..     ...      ...     ...       ...
995    995   369761     3.0         3
996    996  1065126     4.0         3
997    997  1101467     4.0         3
998    998   393413     3.0         3
999    999   478176     4.0         3

[997 rows x 4 columns]


/tmp/ipykernel_12267/110950279.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Movie_Id'] = movie_np.astype(int)
/tmp/ipykernel_12267/110950279.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cust_Id'] = df['Cust_Id'].astype(int)


### Creo matrice di rating

In [470]:
ratings_matrix = df.pivot(index='Cust_Id', columns='Movie_Id', values='Rating')

# Se vuoi gestire eventuali valori mancanti con uno zero, puoi utilizzare il metodo fillna
ratings_matrix = ratings_matrix.fillna(0)

# Se preferisci avere la matrice come array numpy, puoi utilizzare values
#ratings_matrix_array = ratings_matrix.values

print(ratings_matrix)

Movie_Id    1    2    3
Cust_Id                
915       5.0  0.0  0.0
1333      0.0  0.0  4.0
2442      3.0  0.0  0.0
3321      3.0  0.0  0.0
4326      4.0  0.0  0.0
...       ...  ...  ...
2640085   0.0  5.0  0.0
2646060   0.0  0.0  3.0
2646115   0.0  0.0  3.0
2647871   4.0  0.0  0.0
2648861   0.0  3.0  0.0

[985 rows x 3 columns]


### rimuovo le righe nulle

In [444]:
print(ratings_matrix.shape)

(985, 3)


In [445]:
ratings_matrix_support = pd.DataFrame()
for i in ratings_matrix.iterrows():
    val = [not(j) for j in i[1]]
    #print(i[1])
    #print(all(val))
    if not(all(val)):
        ratings_matrix_support = ratings_matrix_support.append(i[1])



/tmp/ipykernel_12267/3021279216.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ratings_matrix_support = ratings_matrix_support.append(i[1])
/tmp/ipykernel_12267/3021279216.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ratings_matrix_support = ratings_matrix_support.append(i[1])
/tmp/ipykernel_12267/3021279216.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ratings_matrix_support = ratings_matrix_support.append(i[1])
/tmp/ipykernel_12267/3021279216.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ratings_matrix_support = ratings_matrix_support.append(i[1])
/tmp/ipykernel_12267/3021279216.py:7: FutureWarning: The frame.a

In [446]:
#ratings_matrix = ratings_matrix_support.copy()
print(ratings_matrix_support.shape)

(985, 3)


# Creo Test set

In [423]:
ratings_matrix.loc[636262]

Movie_Id
1    1.0
2    1.0
3    0.0
Name: 636262, dtype: float64

In [424]:
y_test = {}
count = 0
for i in ratings_matrix.iterrows():
    if count < 5:
        almenoUno = 0
        for j in range(1,len(i[1])):
            if i[1][j] != 0:
                if almenoUno:
                    y_test[(i[0],j)] = i[1][j]
                    i[1][j] = 0
                    count +=1
                    break
                else:
                    almenoUno= True
            

In [425]:
y_test

{(305344, 2): 1.0,
 (387418, 2): 1.0,
 (515436, 2): 1.0,
 (636262, 2): 1.0,
 (1374216, 2): 1.0}

## Usare user based NN

#### Calcolo similarità

In [472]:
rating_medi = {}
# calcolo rating medio senza considerare gli 0
#for i in ratings_matrix.iterrows():
for i in ratings_matrix.iterrows():
        #print(i[0])
        count = 0
        countn = 0
        for j in i[1].values:
            if j != 0:
                  count += j
                  countn += 1
        if countn != 0:
                rating_medi[i[0]] = count/countn
        else:
                rating_medi[i[0]] = 0
#print(rating_medi[1333])
print(rating_medi)

{915: 5.0, 1333: 4.0, 2442: 3.0, 3321: 3.0, 4326: 4.0, 6689: 4.0, 11409: 5.0, 11589: 3.0, 13651: 3.0, 14756: 4.0, 14924: 5.0, 16272: 4.0, 21722: 4.0, 21983: 4.0, 24344: 4.0, 30245: 5.0, 30878: 4.0, 31913: 4.0, 34907: 3.0, 38052: 3.0, 41371: 5.0, 41422: 4.0, 42921: 3.0, 42930: 3.0, 44783: 3.0, 44937: 5.0, 45117: 5.0, 51230: 4.0, 51334: 4.0, 52540: 1.0, 54774: 4.0, 55016: 3.0, 57961: 4.0, 59052: 2.0, 60343: 5.0, 65932: 3.0, 66414: 5.0, 67315: 4.0, 68033: 4.0, 68959: 3.0, 69809: 5.0, 77266: 2.0, 78931: 5.0, 79160: 4.0, 87113: 2.0, 93986: 5.0, 94565: 4.0, 99400: 5.0, 101597: 5.0, 104768: 5.0, 105086: 5.0, 108052: 4.0, 109089: 5.0, 115498: 3.0, 120491: 5.0, 121073: 5.0, 121318: 4.0, 121456: 4.0, 122197: 1.0, 124105: 4.0, 134001: 4.0, 136106: 3.0, 143274: 3.0, 145873: 3.0, 147386: 5.0, 151004: 5.0, 153249: 4.0, 155164: 4.0, 156078: 5.0, 162854: 4.0, 166041: 4.0, 166100: 4.0, 172264: 4.0, 173930: 4.0, 175763: 4.0, 181592: 4.0, 183215: 4.0, 183903: 5.0, 188416: 3.0, 188613: 4.0, 190418: 3.0, 1

In [427]:
# Definire la funzione per il calcolo della similarità tra utenti utilizzando la person coefficent

def user_similarity(ratings,user_target,id_user_target):
    similarità = {}
    # scorro UTENTI
    for i in ratings.iterrows():
        # se l'UTENTE è diverso dall'utente target
        if i[0] != id_user_target:
            # calcolo similarità
            
            numeratore = 0
            denominatoreP1 = 0
            denominatoreP2 = 0
            for j in i[1].keys():
                #if user_target[j] != 0:
                numeratore += (i[1][j]-rating_medi[i[0]])*(user_target[j]-rating_medi[id_user_target])
                
                denominatoreP1 += (i[1][j]-rating_medi[i[0]])**2
                
                denominatoreP2 +=(user_target[j]-rating_medi[id_user_target])**2
            
            sim = 0
            if denominatoreP1 !=0 and denominatoreP2 != 0:
                sim = numeratore/(math.sqrt(denominatoreP1)*math.sqrt(denominatoreP2))
            #print(i[0],id_user_target)
            similarità[(i[0],id_user_target)] = sim
    return similarità

### pre-processing sulla similarità

In [428]:
similarita_coppie = {}
for i in ratings_matrix.iterrows():
    #print(i)
    # Calcolare la similarità tra utenti
    similarita_coppie.update(user_similarity(ratings_matrix,i[1],i[0]))

In [84]:
utenteSimileUno = ratings_matrix.loc[261]
utenteSimileDue = ratings_matrix.loc[2044720]
print(utenteSimileUno)
print(utenteSimileDue)
print(user_target)


Movie_Id
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    4.0
Name: 261, dtype: float64
Movie_Id
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     1.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    1.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    4.0
Name: 2044720, dtype: float64
Movie_Id
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    3.0
Name: 2442, dtype

In [429]:
# FACCIO UNA SOGLIO K DI VICINATO SULLA BASE DELLA SIMILARITÀ
def calcoloSimilaritaConSoglia(similarita):
    sim_soglia = {}
    for i in similarita:
        if similarita[i] > 0.95:
            sim_soglia[i] = similarita[i]
    return sim_soglia

#### Prediction

In [430]:
# Definire una funzione per la predizione delle valutazioni utilizzando User-Based Collaborative Filtering
def predict_user_based(item,user_similarities,ratings_matrix,rating_medi,id_user_target):
    # Calcolare le valutazioni previste
    num = 0
    den = 0
    pred_ratings = 0
    # verifico che per quell'item il rating sia 0
    #if ratings_matrix.loc[id_user_target][item] == 0:
    for i in user_similarities:
        #if ratings_matrix.loc[i[0]].values[item] != 0:
        num += (user_similarities[i]* (ratings_matrix.loc[i[0]][item]-rating_medi[i[0]]))
        den += abs(user_similarities[i])
    if den != 0:
        pred_ratings = rating_medi[id_user_target] + (num / den)
    else:
        pred_ratings = rating_medi[id_user_target]
    print(pred_ratings)
    return pred_ratings
    #else:
    #    return ratings_matrix.loc[id_user_target][item]

# Ottenere le previsioni per tutte le valutazioni
#predicted_ratings_user_based = predict_user_based(id_item_target, sim_soglia,ratings_matrix,rating_medi,id_user_target)
#print(f'Rating per l item {id_item_target} è di {predicted_ratings_user_based} per l utente {id_user_target}')


### Ranking

In [431]:
# Ottenere le previsioni per tutte le valutazioni
y_pred = {}

for i in y_test:
    print(f'{i} starting...')
    # vettore rating utente target
    print(f'user target processing...')
    user_target = ratings_matrix.loc[i[0]]
    
    print('sim_soglia processing...')
    # SIMILARITÀ SOPRA CERTA SOGLIA
    sim_soglia = calcoloSimilaritaConSoglia(similarita_coppie)
    # predico rating test
    print('predicting...')
    predicted_ratings_user_based = predict_user_based(i[1], sim_soglia, ratings_matrix,rating_medi,i[0])   
    if predicted_ratings_user_based < 0:
        predicted_ratings_user_based = 0
    # rating predetto
    print('Appending...')
    y_pred[i]=round(predicted_ratings_user_based,2)


(305344, 2) starting...
user target processing...
sim_soglia processing...
predicting...
-2.5602759772411265
Appending...
(387418, 2) starting...
user target processing...
sim_soglia processing...
predicting...
-2.5602759772411265
Appending...
(515436, 2) starting...
user target processing...
sim_soglia processing...
predicting...
-2.5602759772411265
Appending...
(636262, 2) starting...
user target processing...
sim_soglia processing...
predicting...
-2.5602759772411265
Appending...
(1374216, 2) starting...
user target processing...
sim_soglia processing...
predicting...
-1.5602759772411265
Appending...


In [506]:
print(f'y_pred:{y_pred}')
print(f'y_test:{y_test}')

y_pred:{(305344, 2): 0, (387418, 2): 0, (515436, 2): 0, (636262, 2): 0, (1374216, 2): 0}
y_test:{(305344, 2): 1.0, (387418, 2): 1.0, (515436, 2): 1.0, (636262, 2): 1.0, (1374216, 2): 1.0}


In [505]:
# i'm deleting the rate predicted for evaluationg sim_soglia
""" for i in range(len(y_true)):rating_medi[id_user_target]
    if y_true[i] == 0:
        y_pred[i] = 0 """

# Calcola le metriche aggiuntive

precision = precision_score([round(y_test[pred]) for pred in y_test], [round(y_pred[pred]) for pred in y_pred], average='micro')
recall = recall_score([round(y_test[pred]) for pred in y_test], [round(y_pred[pred]) for pred in y_pred], average='micro')
f1 = f1_score([round(y_test[pred]) for pred in y_test], [round(y_pred[pred]) for pred in y_pred], average='micro')
#nmae = accuracy.mae(predictions) / (max(y_true) - min(y_true))
#mae = accuracy.mae(y_pred)

def mae(true,pred):
    n = len(true)
    tot_val = 0
    for i in true:
        val = abs(pred[i]-true[i])
        tot_val += val
    print(f'MAE:{1-(tot_val/n)}')

def Nmae(true,pred):
    n = len(true)
    tot_val = 0
    max = 0
    min = 9999
    for i in true:
        if true[i] < min:
            min = true[i]
        if true[i] > max:
            max = true[i]
        val = abs(pred[i]-true[i])
        tot_val += val
    print(f'NMAE:{1-(tot_val/n)}')

def rmse(true,pred):
    n = len(true)
    tot_val = 0
    for i in true:
        val = (pred[i]-true[i])**2
        tot_val += val
    rapp = tot_val/n
    print(f'RMSE:{1-math.sqrt(rapp)}')

mae(y_test,y_pred)
Nmae(y_test,y_pred)
rmse(y_test,y_pred)

# Stampa i risultati
print(f'Precision:',precision)
print(f'Recall:',recall)
print(f'F1 Score:',f1)
#print(f'NMAE:',nmae)
#print(f'MAE:',mae)

MAE:0.0
NMAE:0.0
RMSE:0.0
Precision: 0.0
Recall: 0.0
F1 Score: 0.0


In [16]:
# Load the MovieLens dataset
reader = Reader()
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']][:],reader)

# Create a user-based collaborative filtering model
sim_options = {
    'name': 'cosine',
    'user_based': True  # Compute user similarity
}

# Initialize the KNNBasic algorithm
model = KNNBasic(sim_options=sim_options)

# Perform cross-validation
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...


MemoryError: Unable to allocate 275. GiB for an array with shape (192011, 192011) and data type float64

In [17]:
# Load the dataset
#data = Dataset.load_builtin('ml-100k')
# Split the data into a training set and a test set
trainset, testset = train_test_split(data, test_size=0.25)
# Initialize and fit the model
model = KNNBasic(sim_options=sim_options)
model.fit(trainset)
# Make predictions
predictions = model.test(testset)
# Calculate RMSE
rmse = accuracy.rmse(predictions)
print(f'RMSE: {rmse}')

Computing the cosine similarity matrix...


MemoryError: Unable to allocate 257. GiB for an array with shape (185662, 185662) and data type float64

## Usare item based NN 

In [73]:
# Load the MovieLens dataset
reader = Reader()
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']][:],reader)

# Create a user-based collaborative filtering model
sim_options = {
    'name': 'cosine',
    'item_based': True  # Compute user similarity
}

# Initialize the KNNBasic algorithm
model = KNNBasic(sim_options=sim_options)

# Perform cross-validation
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...


MemoryError: Unable to allocate 34.0 GiB for an array with shape (67513, 67513) and data type float64

In [51]:
# Load the dataset
data = Dataset.load_builtin('ml-100k')
# Split the data into a training set and a test set
trainset, testset = train_test_split(data, test_size=0.25)
# Initialize and fit the model
model = KNNBasic(sim_options=sim_options)
model.fit(trainset)
# Make predictions
predictions = model.test(testset)
# Calculate RMSE
rmse = accuracy.rmse(predictions)
print(f'RMSE: {rmse}')

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0166
RMSE: 1.0166491829442186


In [481]:
ratings_matrix_transpose = df.pivot(index='Movie_Id', columns='Cust_Id', values='Rating')

# Se vuoi gestire eventuali valori mancanti con uno zero, puoi utilizzare il metodo fillna
ratings_matrix_transpose = ratings_matrix_transpose.fillna(0)

# Se preferisci avere la matrice come array numpy, puoi utilizzare values
#ratings_matrix_array = ratings_matrix.values

print(ratings_matrix_transpose)

Cust_Id   915      1333     2442     3321     4326     6689     11409    \
Movie_Id                                                                  
1             5.0      0.0      3.0      3.0      4.0      0.0      0.0   
2             0.0      0.0      0.0      0.0      0.0      0.0      5.0   
3             0.0      4.0      0.0      0.0      0.0      4.0      0.0   

Cust_Id   11589    13651    14756    ...  2630337  2630797  2631796  2632461  \
Movie_Id                             ...                                       
1             3.0      3.0      4.0  ...      5.0      5.0      4.0      0.0   
2             0.0      0.0      0.0  ...      0.0      0.0      0.0      0.0   
3             0.0      0.0      0.0  ...      0.0      0.0      0.0      3.0   

Cust_Id   2635437  2640085  2646060  2646115  2647871  2648861  
Movie_Id                                                        
1             4.0      0.0      0.0      0.0      4.0      0.0  
2             0.0      5.0  

In [482]:
# Definire la funzione per il calcolo della similarità tra utenti utilizzando la person coefficent
def item_similarity(ratings,item_target,id_item_target):
    similarita_transpose = {}
    # scorro UTENTI
    for i in ratings.iterrows():
        # i riga film con rating utenti
        # se l'ITEM è diverso dall'item target
        if i[0] != id_item_target:
            # calcolo similarità
            #print(i[0])
            numeratore = 0
            denominatoreP1 = 0
            denominatoreP2 = 0
            for j in i[1].keys():
                #print(rating_medi[j])
                # j utente
                #if i[1][j] != 0 and item_target[j] != 0:
                numeratore += (i[1][j]-rating_medi[j])*(item_target[j]-rating_medi[j])
                denominatoreP1 += (i[1][j]-rating_medi[j])**2
                denominatoreP2 +=(item_target[j]-rating_medi[j])**2
            sim = 0
            if denominatoreP1 != 0 and denominatoreP2 != 0:
                sim = numeratore/(math.sqrt(denominatoreP1)*math.sqrt(denominatoreP2))
            similarita_transpose[(i[0],id_item_target)] = sim

    return similarita_transpose
# Calcolare la similarità tra utenti
#item_similarity(ratings_matrix_transpose,item_target)
#print(similarita_transpose)

In [483]:
similarita_coppie_transpose = {}
for i in ratings_matrix_transpose.iterrows():
    # print(i)
    # Calcolare la similarità tra utenti
    similarita_coppie_transpose.update(item_similarity(ratings_matrix_transpose,i[1],i[0]))

In [484]:
similarita_coppie_transpose

{(2, 1): 0.4849558230121553,
 (3, 1): 0.2483191294176117,
 (1, 2): 0.4849558230121553,
 (3, 2): 0.7227169254225819,
 (1, 3): 0.2483191294176117,
 (2, 3): 0.7227169254225819}

In [118]:
# FACCIO UNA SOGLIO K DI VICINATO SULLA BASE DELLA SIMILARITÀ
sim_soglia_transpose = {}
for i in similarita_transpose:
    if similarita_transpose[i] > 0.0:
        sim_soglia_transpose[i] = similarita_transpose[i]
print(sim_soglia_transpose)

{(2, 1): 0.045871176736922314}


In [486]:
# prendo solo gli item votati dall'utente target
def similaritaSoglia(similarita_transpose):
    sim_soglia_transpose = {}
    for i in similarita_transpose:
        if user_target[i[0]] > 0.5:
            sim_soglia_transpose[i] = similarita_transpose[i]
    return sim_soglia_transpose

In [499]:
# Definire una funzione per la predizione delle valutazioni utilizzando User-Based Collaborative Filtering
def predict_item_based(user,item_similarities,ratings_matrix_transpose,id_item_target):
    # Calcolare le valutazioni previste
    num = 0
    den = 0
    #if ratings_matrix_transpose.loc[id_item_target][user] == 0:
    for i in item_similarities:
        #print(i)
        if ratings_matrix_transpose.loc[i[0]][user] != 0:
            num += (item_similarities[i]* (ratings_matrix_transpose.loc[i[0]][user]))
            den += item_similarities[i]
    pred_ratings = 0
    if den != 0:
        pred_ratings = (num / den)
    return pred_ratings
    #else:
        #return ratings_matrix_transpose.loc[id_item_target][user]

# Ottenere le previsioni per tutte le valutazioni
#predicted_ratings_user_based = predict_item_based(id_item_target, similarita_transpose,ratings_matrix_transpose,id_item_target)
#print(f'Rating per l item {id_item_target} è di {predicted_ratings_user_based} per l utente {id_user_target}')


In [501]:
# Ottenere le previsioni per tutte le valutazioni
y_pred_transpose = {}

for i in y_test:
    print(f'{i} starting...')
    # vettore rating utente target
    print(f'user target processing...')
    item_target = ratings_matrix_transpose.loc[i[1]]
    
    print('sim_soglia processing...')
    # SIMILARITÀ SOPRA CERTA SOGLIA
    sim_soglia_transpose = similaritaSoglia(similarita_coppie_transpose)
    # predico rating test
    print('predicting...')
    predicted_ratings_item_based = predict_item_based(i[0], sim_soglia_transpose, ratings_matrix_transpose,i[1])   
    
    if predicted_ratings_item_based < 0:
        predicted_ratings_item_based = 0
    # rating predetto
    print('Appending...')
    y_pred_transpose[i]=round(predicted_ratings_item_based,2)


(305344, 2) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(387418, 2) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(515436, 2) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(636262, 2) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...
(1374216, 2) starting...
user target processing...
sim_soglia processing...
predicting...
Appending...


In [509]:
print(f'y_pred:{y_pred_transpose}')
print(f'y_true:{y_test}')

y_pred:{(305344, 2): 1.0, (387418, 2): 1.0, (515436, 2): 1.0, (636262, 2): 1.0, (1374216, 2): 2.0}
y_true:{(305344, 2): 1.0, (387418, 2): 1.0, (515436, 2): 1.0, (636262, 2): 1.0, (1374216, 2): 1.0}


In [511]:
# i'm deleting the rate predicted for evaluationg sim_soglia
""" for i in range(len(y_true)):
    if y_true[i] == 0:
        y_pred[i] = 0 """


precision = precision_score([round(y_test[pred]) for pred in y_test], [round(y_pred_transpose[pred]) for pred in y_pred_transpose], average='micro')
recall = recall_score([round(y_test[pred]) for pred in y_test], [round(y_pred_transpose[pred]) for pred in y_pred_transpose], average='micro')
f1 = f1_score([round(y_test[pred]) for pred in y_test], [round(y_pred_transpose[pred]) for pred in y_pred_transpose], average='micro')
#nmae = accuracy.mae(predictions) / (max(y_true) - min(y_true))
#mae = accuracy.mae(y_pred)

def mae(true,pred):
    n = len(true)
    tot_val = 0
    for i in pred:
        val = abs(pred[i]-true[i])
        tot_val += val
    print(f'MAE:{1-(tot_val/n)}')

def Nmae(true,pred):
    n = len(true)
    tot_val = 0
    max = 0
    min = 9999
    for i in pred:
        if true[i] < min:
            min = true[i]
        if true[i] > max:
            max = true[i]
        val = abs(pred[i]-true[i])
        tot_val += val
    print(f'NMAE:{1-(tot_val/n)}')

def rmse(true,pred):
    n = len(true)
    tot_val = 0
    for i in pred:
        val = (pred[i]-true[i])**2
        tot_val += val
    rapp = tot_val/n
    print(f'RMSE:{1-math.sqrt(rapp)}')

mae(y_test,y_pred_transpose)
Nmae(y_test,y_pred_transpose)
rmse(y_test,y_pred_transpose)

# Stampa i risultati
print(f'Precision:',precision)
print(f'Recall:',recall)
print(f'F1 Score:',f1)
#print(f'NMAE:',nmae)
#print(f'MAE:',mae)

MAE:0.8
NMAE:0.8
RMSE:0.5527864045000421
Precision: 0.8
Recall: 0.8
F1 Score: 0.8000000000000002


## Paradigma : User based vs item base

## Matrix factorization: SVD 

In [28]:
reader = Reader()

# get just top 100K rows for faster run time
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']][:], reader)
#data.split(n_folds=3)
trainsetData, testsetData = train_test_split(data, test_size=0.2)

In [29]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'],verbose=True,return_train_measures=True,n_jobs=100)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0534  1.0547  1.0481  1.0482  1.0531  1.0515  0.0028  
MAE (testset)     0.8387  0.8389  0.8348  0.8382  0.8472  0.8396  0.0041  
RMSE (trainset)   0.6774  0.6829  0.6859  0.6744  0.6792  0.6799  0.0041  
MAE (trainset)    0.5364  0.5416  0.5420  0.5358  0.5391  0.5390  0.0026  
Fit time          1.50    1.58    1.54    1.55    1.64    1.56    0.05    
Test time         0.17    0.15    0.11    0.11    0.19    0.15    0.03    


{'test_rmse': array([1.05339768, 1.05468582, 1.04808826, 1.04818772, 1.05307251]),
 'train_rmse': array([0.67739404, 0.68285783, 0.68592011, 0.67438005, 0.6791954 ]),
 'test_mae': array([0.83870176, 0.83890415, 0.83480825, 0.83816799, 0.84720285]),
 'train_mae': array([0.53635112, 0.54164428, 0.54198087, 0.5358227 , 0.53910241]),
 'fit_time': (1.5020723342895508,
  1.5849971771240234,
  1.5369455814361572,
  1.546156406402588,
  1.6409692764282227),
 'test_time': (0.1729419231414795,
  0.1542515754699707,
  0.11395001411437988,
  0.11310744285583496,
  0.18653631210327148)}

## NN

In [30]:
xTrain = pd.DataFrame({'Movie_Id': df['Movie_Id'], 'Cust_Id': df['Cust_Id']})
yTrain = pd.DataFrame({'Rating': df['Rating']})
print(xTrain.head())
print(yTrain.head())

   Movie_Id  Cust_Id
1         1  1488844
2         1   822109
3         1   885013
4         1    30878
5         1   823519
   Rating
1     3.0
2     5.0
3     4.0
4     4.0
5     3.0


In [151]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate


# Input layer
ratings_input = Input(shape=(None,2))

# Embedding layers
dense0 = Dense(32, activation='relu')(ratings_input)

# Dense layers
#dense1 = Dense(128, activation='relu')(dense0)
dense2 = Dense(64, activation='relu')(dense0)

# Output layer
output = Dense(6,activation='softmax')(dense2)

# Model
model = Model(inputs=ratings_input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')



In [153]:
model.fit(xTrain,yTrain, epochs=50, batch_size=128)

Epoch 1/50
 18/782 [..............................] - ETA: 2s - loss: 12.6623

782/782 [==============================] - 1s 1ms/step - loss: 12.7181
Epoch 2/50
782/782 [==============================] - 1s 1ms/step - loss: 12.7181
Epoch 3/50
782/782 [==============================] - 1s 1ms/step - loss: 12.7181
Epoch 4/50
782/782 [==============================] - 1s 1ms/step - loss: 12.7181
Epoch 5/50
782/782 [==============================] - 1s 1ms/step - loss: 12.7181
Epoch 6/50
782/782 [==============================] - 1s 1ms/step - loss: 12.7181
Epoch 7/50
782/782 [==============================] - 1s 1ms/step - loss: 12.7181
Epoch 8/50
782/782 [==============================] - 1s 1ms/step - loss: 12.7181
Epoch 9/50
782/782 [==============================] - 1s 1ms/step - loss: 12.7181
Epoch 10/50
782/782 [==============================] - 1s 1ms/step - loss: 12.7181
Epoch 11/50
782/782 [==============================] - 1s 1ms/step - loss: 12.7181
Epoch 12/50
782/782 [==============================] - 1s 1ms/step - loss: 12.7181
Epoch 13/50
782/782 [===

In [156]:
y_pred = model.predict(xTrain)


3125/3125 [==============================] - 3s 921us/step


" mae = np.mean(np.abs(y_pred - yTrain))\nprint(f'MAE: {mae}') "

In [163]:
print(y_pred)
y_pred_custom = []
for i in y_pred:
    val = -1
    index = -1
    for j in range(len(i)):
        if i[j] > val:
            val = i[j]
            index = j
    y_pred_custom.append(index)
y_pred_custom = pd.DataFrame(y_pred_custom)
print(y_pred_custom)


[[0.         0.         0.         0.         0.99999994 0.        ]
 [0.         0.         0.         0.         0.99999994 0.        ]
 [0.         0.         0.         0.         0.99999994 0.        ]
 ...
 [0.         0.         0.         0.         0.99999994 0.        ]
 [0.         0.         0.         0.         1.         0.        ]
 [0.         0.         0.         0.         1.         0.        ]]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 

In [161]:
mae = np.mean(np.abs(y_pred_custom - yTrain))
print(f'MAE: {mae}')


ValueError: Unable to coerce to Series, length must be 1: given 99970

In [159]:
mae = np.mean(np.abs(y_pred - yTrain))
print(f'MAE: {mae}')
loss = model.evaluate(xTrain, yTrain)
print(f'Loss on test set: {loss}')


ValueError: Unable to coerce to DataFrame, shape must be (99970, 1): given (99970, 6)